# Learning the Structure of Generative Models

In this notebook, we'll use structure learning to find the dependency structure of a generative model. You can do this for any label matrix!

See the [blog post](https://hazyresearch.github.io/snorkel/blog/structure_learning.html) or the [paper](https://arxiv.org/abs/1703.00854) for more details.

### Generating Some Data

We'll generate some data from a known model of noisy labels in which two pairs of labeling functions are correlated.

In [1]:
from snorkel.learning import GenerativeModelWeights
from snorkel.learning.structure import generate_label_matrix

weights = GenerativeModelWeights(10)
for i in range(10):
    weights.lf_accuracy[i] = 1.0
weights.dep_similar[0, 1] = 0.5
weights.dep_similar[2, 3] = 0.5

y, L = generate_label_matrix(weights, 10000)

### Structure Learning
`L` is the label matrix produced by a `LabelManager`.
A few notes:
* The `deps` object is a collection of tuples specifying which labeling functions are related by which types of dependencies.
* The keyword argument `threshold` is a positive float that indicates how strong the dependency has to be for it to be returned in the collection. Too many dependencies? Turn it up. Too few? Turn it down.
* By default, the `DependencySelector` looks for pairwise correlations between labeling functions. Pass the keyword argument `higher_order=True` to the `select` method to also look for reinforcing and fixing dependencies (described in the [data programming paper](https://arxiv.org/abs/1605.07723)).

In [2]:
from snorkel.learning.structure import DependencySelector
ds = DependencySelector()
deps = ds.select(L, threshold=0.05)
print deps
assert deps == set([(0, 1, 0), (2, 3, 0)])

set([(0, 1, 0), (2, 3, 0)])


### Using the Learned Structure

To incorporate the selected dependencies into your generative model, just pass them in as a keyword argument:

In [3]:
from snorkel.learning import GenerativeModel
gen_model = GenerativeModel()
gen_model.train(L, deps=deps)

Inferred cardinality: 2


In [4]:
print gen_model.weights.lf_accuracy

[ 0.99759367  1.03627493  0.9562079   1.02648033  1.04138374  1.01789283
  0.9971977   1.0053931   1.00280444  0.98741134]
